# SciML SANUM2024
# Lab 4: Neural Networks and Lux.jl

In this lab we introduce neural networks as implemented in Lux.jl.
A neural network (NN) is in some sense just a function with many parameters
in a way that facilitates computing gradients with respect to these parameters.
That is: it is at its core a way of book-keeping a heavily parameterised function.
It is constructed by composing basic building blocks usually built from linear
algebra operations, combined with simple _activator functions_.
Here we look at the simplest case and see how the paremeters in a NN can be chosen to
solve optimisation problems.

**Learning Outcomes**
1. Single-layer neural networks and activation functions.
2. Creating deeper networks as a `Chain`.
3. Training neural networks by simple optimisation.

In [ ]:
using Lux, Random, Optimisers, Zygote, Plots, LinearAlgebra, Test

## 4.1 Single layer neural networks

We begin with a single-layer neural network without an activator
function. This happens to be precisely maps of the form
$$
𝐱 ↦ A𝐱 + 𝐛
$$
where $A ∈ ℝ^{m × n}$ and $𝐛 ∈ ℝ^n$. The space of such maps is
modelled by the `Dense` type, where the `weight` corresponds to $A$
and the `bias` corresponds to $𝐛$. Here we see a simple example
of constructing the model (the space of all such maps) and evaluating
a specific map by specifying the paramters:

In [ ]:
m,n = 5,4

model = Dense(n => m) # represents

A = randn(5,4)
b = randn(5)
x = randn(4)
const NOSTATE = NamedTuple() # no state
val,newst = model(x, (weight=A, bias=b), NOSTATE) # returns the output of the map and the "state", which we ignore

@test val ≈ A*x + b # our model with these parameters is just A*x + b

An important feature is that we can compute gradients with respect to parameters of functions of our
model. Before we looked at the case where
we differentiated with respect to vectors but a power feature in Zygote is it works for all types like named-tuples.

In [ ]:
ps = (weight=A, bias=b)
ps_grad = gradient(p -> sum(model(x, p, NOSTATE)[1]), ps)[1] # returns a named tuple containing the gradients

Because our Neural Network at this stage is linear in the paremeters the gradient is actually quite simple: eg the partial derivative with
respect to $A[k,j]$ will just be $x[j]$ and the derivative with respect to $b[k]$ will just be $1$. Thus we get:

In [ ]:
@test ps_grad[:weight] ≈ ones(5) * x'
@test ps_grad[:bias] ≈ ones(5)

Going beyond basic linear algebra, we can apply an "activator" function $f$ to each
entry of the map, to represent maps of the form:
$$
𝐱 ↦ f.(A𝐱 + 𝐛)
$$
Where we use the Julia-like broadcast notation to mean entrywise application.
The classic in ML is the `relu` function which is really just $\max(0,x)$:

In [ ]:
x = range(-1,1, 1000)
plot(x, relu.(x); label="relu")

We can incorporate this in our model as follows:

In [ ]:
model = Dense(4 => 5, relu)
model(x, (weight = A, bias=b), NOSTATE)[1]

And we can differentiate:

In [ ]:
ps = (weight=A, bias=b)
ps_grad = gradient(p -> sum(model(x, p, NOSTATE)[1]), ps)[1] # returns a named tuple containing the gradients

**Problem 1** Derive the forumula  for the gradient of the model with an activator function and compare it with
the numerical result just computed. Hint: The answer depends on the output value.

Let's see an example directly related to a classic numerical analysis problem: approximating
functions by a continuous piecewise affine
function, as done in the Trapezium rule. Our model corresponds to a sum of weighted and shifted `relu` functions:
$$
p_{𝐚,𝐛}(x) := ∑_{k=1}^n {\rm relu}(a_k x + b_k)
$$
We note that this is a sum of positive convex functions so only useful for approximating positive convex functions
(we will generalise this later).  Thus we want to choose the paremeters to fit data generated by a positive convex function,
e.g., $f(x) = 1 - \exp(-x^2)$. Here we first generate "training data" which means the samples of the function on a grid.
We want our data to be a $1×n$ matrix show

In [ ]:
n = 100
x = range(-1, 1; length = n)
y = exp.(x)

Our one-layer neural network (before the summation) is
$$
  relu.(𝐚x + 𝐛)
$$
which corresponds to a simple dense layer with `relu` activation.
We then sum over the output of this to get the model
$$
  [1,…,1]^⊤ relu.(𝐚x + 𝐛)
$$
In our case `x` will be a vector containing the grid we sample on
but we first need to transpose it to be a $1 × n$ matrix, which will apply the NN
to each grid point. We can then sum over the columns to get the value of the model with the given
paramaters at the grid points.

In [ ]:
nn = Dense(1 => n, relu)
function summation_model(nn, x, ps)
    Y,st = nn(x', ps, NOSTATE) # k-th column contains relu.(𝐚x[k] + 𝐛)
    vec(sum(Y; dims=1)) # sums over the columns
end

We want to choose the paramters to minimise a loss function. Here we
just wish to minimise the 2-norm erro which we can write as follow:

In [ ]:
function convex_regression_loss(nn, ps, (x,y))
    ỹ = summation_model(nn, x, ps)
    norm(ỹ - y) # 2-norm error
end

We could try to use Optimizers.jl directly on this function,
but Lux.jl provides a wrapper around the optimisation routine that is convenient for ML.
We need to make sure our loss function fits the right interface:
in addition to th eloss we also need to return the state and the "computed statistics",

In [ ]:
#  which in our case is empty.

convex_regression_loss_lux(nn, ps, st, (x,y)) = convex_regression_loss(nn, ps, (x,y)), st, ()

We can now run our optimisation routine. Here we run over 250 "epochs": this specifies the number of times
we do optimisation. We wrap this in a simple function that we can reuse for other examples:

In [ ]:
rng = MersenneTwister()
opt = Adam(0.03f0)
tstate = Lux.Experimental.TrainState(rng, nn, opt)

function train(lossfunc, data, tstate, N)
    for epoch in 1:N
        grads, loss, stats, tstate = Lux.Training.compute_gradients(AutoZygote(), lossfunc, data, tstate)
        if epoch % 50 == 1 || epoch == N
            @show epoch, loss
        end
        tstate = Lux.Training.apply_gradients(tstate, grads)
    end
    tstate
end

tstate = train(convex_regression_loss_lux, (x,y), tstate, 250)

We can compare our approximation to the data and see we have successfully trained the simple neural network:

In [ ]:
plot(x, y)
plot!(x, summation_model(nn, x, tstate.parameters))

**Problem 2**  Replace `relu` in the activation function with a smooth `tanh` function and plot
the result. Is the approximation as accurate? What if you increase the number of epochs?

In [ ]:
# TODO: setup a neural network with a different activation

## 4.2 Multiple layer neural networks

An effective neural network will have more than one layer. A simple example is if we want to go beyond
convex functions. Rather than simply summing over the neural network we can allow different weights,
giving us the model
$$
  𝐜^⊤ relu.(𝐚x + 𝐛) + d.
$$
Or we can think of $C = 𝐜^⊤$ as a $1 × n$ matrix. This is in fact a composition of two simple layers, the first being
$$
 x ↦ relu.(𝐚x + 𝐛)
$$
and the second being
$$
 𝐱 ↦ C 𝐱 + d
$$
i.e., they are both `Dense` layers just with different dimensions and different activation functions (`relu` and `identity`).
We can create such a composition using the `Chain` command:

In [ ]:
n = 100
model = Chain(Dense(1 => n, relu), Dense(n => 1))

Here the parameters are nested. For example, we can create the relevant parameters as follows:

In [ ]:
𝐚,𝐛 = randn(n,1),randn(n)
𝐜,d = randn(n),randn(1)

st = (layer_1 = NOSTATE, layer_2 = NOSTATE) # each layer has its own state
ps = (layer_1 = (weight = 𝐚, bias = 𝐛), layer_2 = (weight = 𝐜', bias = d))

@test model([0.1], ps, st)[1] ≈ 𝐜'*relu.(𝐚*0.1 + 𝐛) + d

We can plot the model evaluated at the gri to see that it is indeed (probably) no longer convex:

In [ ]:
plot(x, vec(model(x', ps, st)[1]))

We now choose the parameters to fit data. Let's generate data for a non-convex function:

In [ ]:
x = range(-1, 1; length = n)
y = sin.(3x).*exp.(x)
plot(x,y)

We will fit this data by minimising the 2-norm with a different model:

In [ ]:
function regression_loss(model, ps, st, (x,y))
    ỹ = vec(model(x', ps, st)[1])
    norm(ỹ - y) # 2-norm error
end

regression_loss_lux(model, ps, st, (x,y)) = regression_loss(model, ps, st, (x,y)), st, ()

We now run the optimiser:

In [ ]:
opt = Adam(0.03f0)
tstate = Lux.Experimental.TrainState(rng, model, opt)

tstate = train(regression_loss_lux, (x,y), tstate, 1000)


plot(x,y)
plot!(x, vec(model(x', tstate.parameters, tstate.states)[1]))

 It does OK but is still not particularly impressive. The real power in neural networks is their approximation power
increases as we add more layers. Here let's try an example with 3-layers.

In [ ]:
model = Chain(Dense(1 => n, relu), Dense(n => n, relu), Dense(n => 1))
tstate = Lux.Experimental.TrainState(rng, model, opt)

tstate = train(regression_loss_lux, (x,y), tstate, 1000)
plot!(x, vec(model(x', tstate.parameters, tstate.states)[1]))

**Problem 3** Add a 4th layer and 5th layer, but not all involving square matrices, and compare the `relu` and `tanh`
activation functions. Can you choose the size of the layers to
match the eyeball norm? Hint: the answer might be "no" 😅

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*